In [1]:
# Version Pablo Izquierdo del dia 09/06/2025

from ultralytics import YOLO
import cv2

# Cargar modelo YOLOv11
model = YOLO("yolo11n.pt") # Asegúrate de tener este archivo en el directorio

# Ruta del video de entrada
path = 'C:/Users/gtoma/Master_AI_Aplicada/UEM_Master_AI_07042025/UEM_Trabajo/Sesiones_UEM/Manuel_Garcia_VISION/Actividad_3_YOLOV11/'
video_path = path + 'comma_small.mp4' # Cambia el nombre del video según sea necesario

# Inicializar captura de video
cap = cv2.VideoCapture(video_path)
merged_tracks = {} # Diccionario para almacenar trayectorias

# Obtener propiedades del video
width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
fps = cap.get(cv2.CAP_PROP_FPS) or 30 # FPS estimado si es 0

# Configurar escritor de video
out = cv2.VideoWriter(path+'tracking_coches_test.avi',cv2.VideoWriter_fourcc(*'XVID'), fps, (width, height))
print("Procesando video...")
while cap.isOpened():
    ret, frame = cap.read()
    if not ret:
        break

    # Ejecutar detección y seguimiento en el fotograma actual
    results = model.track(frame, persist=True, conf=0.5, verbose=False)[0]
    boxes = results.boxes
    unique_ids = set()
    for box in boxes:
        if int(box.cls[0]) not in [2, 5, 7] or box.id is None:
            continue
        track_id = int(box.id[0])
        x1, y1, x2, y2 = map(int, box.xyxy[0])
        center = ((x1 + x2) // 2, (y1 + y2) // 2)
        unique_ids.add(track_id)

        # Almacenar trayectoria
        if track_id not in merged_tracks:
            merged_tracks[track_id] = []
        merged_tracks[track_id].append(center)

        # Dibujar caja delimitadora en rojo
        cv2.rectangle(frame, (x1, y1), (x2, y2), (0, 0, 255), 2)
        cv2.putText(frame, f'ID {track_id}', (x1, y1 - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.6, (0, 0, 255), 2)

        # Dibujar trayectoria en blanco
        for pt in merged_tracks[track_id]:
            cv2.circle(frame, (int(pt[0]), int(pt[1])), 2, (255, 255, 255), -1)
    
    # Mostrar el fotograma procesado (opcional)
    cv2.imshow("Detección en video", frame)

    # Guardar el fotograma en el video de salida
    out.write(frame)

    # Salir con la tecla 'q'
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

cap.release()
out.release()
cv2.destroyAllWindows()
print("Procesamiento completado. Video guardado como tracking_coches.avi")



Procesando video...
Procesamiento completado. Video guardado como tracking_coches.avi
